In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split as split
from sklearn.utils import shuffle

In [2]:
# Features Selection
dataset = shuffle(pd.read_csv('student-mat.csv'))
Y = dataset[['romantic']]
X = dataset[['sex', 'age', 'famsize', 'traveltime', 'studytime', 'activities', 'goout', 'famrel']]
Y['romantic'][Y['romantic']=='no'] = 0
Y['romantic'][Y['romantic']=='yes'] = 1
X['sex'][X['sex']=='F'] = 0
X['sex'][X['sex']=='M'] = 1
X['famsize'][X['famsize']=='LE3'] = 0
X['famsize'][X['famsize']=='GT3'] = 1
X['activities'][X['activities']=='no'] = 0
X['activities'][X['activities']=='yes'] = 1
X = X.apply(pd.to_numeric)
X.describe()

/home/wermarter/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:4702: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/wermarter/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/wermarter/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/wermarter/an

,sex,age,famsize,traveltime,studytime,activities,goout,famrel
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,0.473418,16.696203,0.711392,1.448101,2.035443,0.508861,3.108861,3.944304
std,0.499926,1.276043,0.453690,0.697505,0.839240,0.500555,1.113278,0.896659
min,0.000000,15.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,0.000000,16.000000,0.000000,1.000000,1.000000,0.000000,2.000000,4.000000
50%,0.000000,17.000000,1.000000,1.000000,2.000000,1.000000,3.000000,4.000000
75%,1.000000,18.000000,1.000000,2.000000,2.000000,1.000000,4.000000,5.000000
max,1.000000,22.000000,1.000000,4.000000,4.000000,1.000000,5.000000,5.000000


In [3]:
#Splitting into train set, validation set and test set
def train_val_test_split(X, Y, dist):
    X, Y = X.as_matrix().astype(float), Y.as_matrix().astype(float)
    a, b, c = np.array(dist)
    train_x, tmp_x, train_y, tmp_y = split(X, Y, test_size=(b+c)/(a+b+c))
    val_x, test_x, val_y, test_y = split(tmp_x, tmp_y, test_size=c/(b+c))
    return train_x, val_x, test_x, train_y, val_y, test_y
train_x, val_x, test_x, train_y, val_y, test_y = train_val_test_split(X, Y, [7, 1, 2])
len(train_x), len(val_x), len(test_x)

(276, 39, 80)

In [4]:
class MLP_BinClassifier(nn.Module): # Multilayer perceptrons for binary classification
    def __init__(self, 
                 layers=[8, 2, 2, 1], # Layers structure: number of units on each layer
                 lr=0.01, # Learning rate
                 reg_lambda=0.5, # Lambda value for regularization (weight decay)
                 H_activ=F.tanh, # Hidden units' activation function
                 L_activ=F.sigmoid, # Output units' activation function
                ):
        super(MLP_BinClassifier, self).__init__()
        self.lr = lr
        self.reg_lambda = reg_lambda
        self.layers = layers
        self.H_activ = H_activ
        self.L_activ = L_activ
        assert len(layers) >= 3
        assert callable(H_activ) and callable(L_activ)
        # Parameters generator: linears[0], linears[1],..., linears[L]
        params = list()
        for (idx, l) in enumerate(self.layers[:-1]):
             params.append(nn.Linear(l, self.layers[idx+1]))
        self.linears = nn.ModuleList(params)
    def forward(self, X):
        X = X.view(1, -1)
        for i in range(len(self.linears)-1):
            linear = self.linears[i]
            X = self.H_activ(linear(X))
        linear = self.linears[-1]
        X = self.L_activ(linear(X))
        return X[0]

In [5]:
model = MLP_BinClassifier([8, 1, 1, 1, 1, 1], reg_lambda=0)
cuda = torch.cuda.is_available()
if cuda:
    model.cuda()
optimizer = optim.Adam(model.parameters(), lr=model.lr, weight_decay=model.reg_lambda)
def per_example_loss(output, target):
    return -((1-target)*(1-output).log() + target*(output).log()) # negative log likelihood
#criterion = nn.CrossEntropyLoss()
criterion = per_example_loss
def evaluate(X, y):
    model.eval()
    losses = list()
    correct = 0
    for (data, target) in zip(X, y):
        data, target = torch.Tensor(data), torch.Tensor(target)
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target, volatile=True)
        output = model(data)
        loss = criterion(output, target)
        
        losses.append(loss.data[0])
        output, target = 1 if output.data[0] > 0.5 else 0, target.data[0]
        correct += 1 if (output==target) else 0
    return (sum(losses)/len(losses)), correct/len(losses)
def train():
    model.train()
    losses = list()
    for (data, target) in zip(train_x, train_y):
        data, target = torch.Tensor(data), torch.Tensor(target)
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        optimizer.zero_grad() # Empty the gradients
        loss = criterion(output, target)
        loss.backward() # Take the Gradients
        optimizer.step() # Update the model
        losses.append(loss.data[0])
    return sum(losses)/len(losses), evaluate(val_x, val_y)[0] # returns Loss and Cross-Validation Error
def test():
    return evaluate(test_x, test_y) # returns Loss and Accuracy
def predict(X):
    return model(Variable(torch.Tensor(X).cuda())).data[0]


In [6]:
n_epochs = 100
log_interval = 10
allow_early_stop = True
pre_cv_error = float('inf')
patience = 5 # Number of steps with increasing cross-validation error to tolerate before early stopping
tolerated = False
for epoch in range(1, n_epochs+1): # Main loop
    loss, cv_error = train()
    loss_test, accuracy = test()
    if cv_error > pre_cv_error:
        if allow_early_stop:
            if patience == 0:
                print('Early stopping on epoch:', epoch)
                print('Restoring best model from epoch:', best_idx)
                model = torch.load('best_model.pkl')
                break
            else:
                patience -= 1
                tolerated = True
    else:
        patience = 5
        best_idx = epoch
        torch.save(model, 'best_model.pkl') # Can be time-consuming if model is large
    if not tolerated:
        pre_cv_error = cv_error
    # Logging
    if (epoch % log_interval == 0)or(epoch<10):
        print("Epoch: ", epoch)
        print("Training Loss: {}\nCrossValidation Error: {}\nTest Loss: {}\nAccuracy: {}"
              .format(loss, cv_error, loss_test, accuracy))
        print("=======================================")

/home/wermarter/anaconda3/lib/python3.5/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type MLP_BinClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch:  1
Training Loss: 0.6570440369887628
CrossValidation Error: 0.6379277201799246
Test Loss: 0.6839456751942634
Accuracy: 0.5875
Epoch:  2
Training Loss: 0.6248289449275404
CrossValidation Error: 0.6372621273383116
Test Loss: 0.6857256881892682
Accuracy: 0.5875
Epoch:  3
Training Loss: 0.6244365163687347
CrossValidation Error: 0.6372076143056918
Test Loss: 0.6859062511473895
Accuracy: 0.5875
Epoch:  4
Training Loss: 0.6243813960016638
CrossValidation Error: 0.6371690974785731
Test Loss: 0.6860340937972069
Accuracy: 0.5875
Epoch:  5
Training Loss: 0.6243401007807773
CrossValidation Error: 0.637136330207189
Test Loss: 0.6861488737165928
Accuracy: 0.5875
Epoch:  6
Training Loss: 0.6243042342256808
CrossValidation Error: 0.6371082212680426
Test Loss: 0.6862511180341244
Accuracy: 0.5875
Epoch:  7
Training Loss: 0.6242729964247649
CrossValidation Error: 0.6370842143511161
Test Loss: 0.6863411221653223
Accuracy: 0.5875
Epoch:  8
Training Loss: 0.6242458338754765
CrossValidation Error: 0.6

In [7]:
# sex - 0:female or 1:male
# age - ideal range is from 15 to 22
# famsize - family size (0 ~ <=3 or 1 ~ >3)
# traveltime - home to school travel time (1 ~ <15 min, 2 ~ 15-30 min, 3 ~ 30 min - 1 hour, or 4 ~ >1 hour)
# studytime - weekly study time (1 ~ <2 hours, 2 ~ 2-5 hours, 3 ~ 5-10 hours, or 4 ~ >10 hours)
# activities - extra-curricular activities (1 ~ yes or 0 ~ no)
# goout - going out with friends (from 1 ~ very low to 5 ~ very high)
# famrel - quality of family relationships (from 1 ~ very bad to 5 ~ excellent)
sex, age, famsize, traveltime, studytime, activities, goout, famrel = 0, 23, 1, 2, 4, 1, 3, 4
input_data = np.array([sex, age, famsize, traveltime, studytime, activities, goout, famrel]).astype(float)

In [8]:
predict(input_data)*100

34.639376401901245

In [9]:
predict(train_x[2])

0.3463935852050781